In [1]:
import numpy as np

### DECONSTRUCT KEY & VALUE

In [4]:
def key_value(blob):
    key = list(blob)[0]
    value = blob[key]
    
    return key, value

### DECISION MACHINE FOR DATASET LABELS

In [6]:
class decision_machine():
    
    # CALIBRATE QUANTILE THRESHOLDS & RETURN NEW LABELS
    def calibrate(self, dataframe, settings):
        
        # GET LABEL LOG RETURN VALUES
        labels = self.log_returns(dataframe)
        
        # SET QUANTILE THRESHOLDS
        self.lower = labels.quantile(settings['lower'])
        self.upper = labels.quantile(settings['upper'])
        
        return self.vote(labels)
        
    # CONVERT & RETURN NEW LABELS
    def convert(self, dataframe):
        
        # GET LABEL LOG RETURN VALUES
        labels = self.log_returns(dataframe)
        return self.vote(labels)
        
    # DECIDE LABEL TAG VIA VOTE
    def vote(self, labels):
        container = []
        
        # BUY   = 0
        # SELL  = 1
        # HOLD  = 2
        
        # LOOP THROUGH LABELS
        for label in labels:

            # BUY
            if (label < self.lower):
                container.append(0)

            # SELL
            elif label > self.upper:
                container.append(1)

            # HOLD
            else:
                container.append(2)

        return container
    
    # GET LOG RETURN VALUE DF COLUMN
    def log_returns(self, dataframe):
        return np.log(dataframe['label'] / dataframe['label'].shift(1))

### CLONE DATAFRAME & REPLACE LABEL COLUMN

In [5]:
def replace_labels(old_dataframe, labels):
    
    # CLONE & REPLACE
    dataframe = old_dataframe.copy()
    dataframe['label'] = labels
    
    return dataframe

### RENAME DICT KEYS

In [70]:
def rename_settings(old_settings):
    new_settings = {}
    
    for key in old_settings.keys():
        new_settings['model__' + key] = old_settings[key]
        
    return new_settings

### BALANCE PREDICTIONS

In [94]:
def balance_predictions(predictions):
    collection = []
    
    # LOOP THROUGH PREDICTIONS & CHECK LENGTHS
    for values in predictions.values():
        collection.append(len(values))
    
    # FIND THE SMALLEST SIZE
    smallest =  min(collection)
    
    # CUT OFF EXTRA ELEMENTS
    for key in predictions.keys():
        predictions[key] = predictions[key][-smallest:]
    
    return predictions